In [6]:
# Create a dictionary with these
# 0. nose
# 1. left_eye_inner
# 2. left_eye
# 3. left_eve_outer
# 4. right_eye_inner
# 5. right_eye
# 6. right_eye_outer
# 7. left_ear
# 8. right_ear
# 9. mouth_left
# 10. mouth_right
# 11. left_shoulder
# 12. right_shoulder
# 13. left_elbow
# 14. right_elbow
# 15. left_wrist
# 16. right_wrist
# 17. left_pinky
# 18. right_pinky
# 19. left index
# 20. right_index
# 21. left_thumb
# 22. right_thumb
# 23. left_hip
# 24. right_hip
# 25. left_knee
# 26. right_knee
# 27. left_ankle
# 28. right_ankle
# 29. left_heel
# 30. right_heel
# 31. left_foot_index
# 32. right_foot_index

### Creating a dictionary with Body Points for Dimensions Image

In [1]:
body_points={0:("nose",913,362),
             1:("left_eye",933,376),
             2:("left_ear",943,414),
             3:("left_shoulder",795,501),
             4:("right_shoulder",792,353),
             5:("left_elbow",804,597),
             6:("right_elbow",824,223),
             7:("left_palm",815,714),
             8:("right_palm",862,96),
             9:("left_hip",550,487),
             10:("right_hip",374,423),
             11:("left_knee",671,595),
             12:("right_knee",273,551),
             13:("left_ankle",862,711),
             14:("right_ankle",154,701),
             15:("left_heel",830,718),
             16:("right_heel",160,718),
             17:("left_foot_index",974,719),
             18:("right_foot_index",124,715)}

### Reference Dictionary with body parts and index

In [2]:
ref_body_dict=[]
for i in body_points:
    x=body_points[i][0]
    y=i
    ref_body_dict.append((x,y))
ref_body_dict=dict(ref_body_dict)
ref_body_dict

{'nose': 0,
 'left_eye': 1,
 'left_ear': 2,
 'left_shoulder': 3,
 'right_shoulder': 4,
 'left_elbow': 5,
 'right_elbow': 6,
 'left_palm': 7,
 'right_palm': 8,
 'left_hip': 9,
 'right_hip': 10,
 'left_knee': 11,
 'right_knee': 12,
 'left_ankle': 13,
 'right_ankle': 14,
 'left_heel': 15,
 'right_heel': 16,
 'left_foot_index': 17,
 'right_foot_index': 18}

### Drawing points for Dimensions Image

In [3]:
import numpy as np
import cv2
blank=np.zeros((720,1280))
img=cv2.imread("Images_output\yogframe_img+edge+points.jpg")
for i in body_points:
    cv2.circle(img,body_points[i][1:],3,(255,0,0),cv2.FILLED)
# cv2.imshow("dim_yog",img)
# cv2.waitKey()
# cv2.destroyAllWindows()

### Superposing Images

In [21]:
import cv2
import numpy as np

img1 = cv2.imread("dim_pose_sameht.jpg")
img2 = cv2.imread("images\dim_triangles-cropped.png")
img2= cv2.resize(img2,img1.shape[1::-1])
added_image = cv2.addWeighted(img1,0.7,img2,0.3,0)


cv2.imshow('Blended Image',added_image)
# cv2.imwrite("Comparision with ground truth\yogframe_dimyog_cropped.jpg",added_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Pose detection for Inferenced Yoga Image

In [4]:
import mediapipe as mp
import cv2
import time
import numpy as np
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # Landmarks

img = cv2.imread(r'images\yogframe.jpg')
imageWidth, imageHeight = img.shape[:2]
imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
blank = np.zeros(img.shape) # Blank image
results = pose.process(imgRGB)

if results.pose_landmarks:
    mpDraw.draw_landmarks(blank, results.pose_landmarks, mpPose.POSE_CONNECTIONS) # draw landmarks on blank
    landmarks = results.pose_landmarks.landmark


# cv2.imshow("Image", img)
# cv2.imshow("blank",blank)
# # cv2.imwrite('Images_output\yogframe_points.jpg',blank)
# cv2.waitKey()
# cv2.destroyAllWindows()

### Getting Body Points in List from Pose detection 

In [5]:
comb_list=[]
for id, lm in enumerate(results.pose_landmarks.landmark):
    h, w,c = img.shape
    # print(id, lm)
    cx, cy = int(lm.x*w), int(lm.y*h)
    comb_list.append((cx,cy))

comb_list=dict(enumerate(comb_list))

In [6]:
req_points=[0,2,7,11,12,13,14,15,16,23,24,25,26,27,28,29,30,31,32]
req_body_points=[]
for i in comb_list:
    if i in req_points:
        req_body_points.append(comb_list[i])

req_body_points=dict(enumerate(req_body_points))


In [7]:
# Function for Eucledian Line distance
import math
def l_dist(x1,y1,x2,y2):
    sq1=(x2-x1)**2
    sq2=(y2-y1)**2
    res=int(math.sqrt(sq1 + sq2))
    return res

In [8]:
dist=[]
for i in body_points:
    a=req_body_points[i][0]
    b=req_body_points[i][1]
    c=body_points[i][1]
    d=body_points[i][2]
    dist.append(l_dist(a,b,c,d))
len(dist)

19

In [9]:
# function for drawing lines between two points
import cv2
def l_func(img,x1,y1,x2,y2):
    cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)


In [10]:
import cv2
img=cv2.imread("Comparision with ground truth\yogframe_dimyog.jpg")
for i in req_body_points:
    a=req_body_points[i][0]
    b=req_body_points[i][1]
    c=body_points[i][1]
    d=body_points[i][2]
    p=int((a+c)/2)
    q=int((b+d)/2)
    distance=str(dist[i])
    l_func(img,a,b,c,d)
    cv2.putText(img,distance,(p,q),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
# cv2.imshow('Ima',img)
# # cv2.imwrite("Comparision with ground truth\yogframe_dimyog_ptsdist.jpg",img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [11]:
ref_body_dict.keys()

dict_keys(['nose', 'left_eye', 'left_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_palm', 'right_palm', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index'])

In [12]:
ref_datum=[i for i in ref_body_dict.keys() if i in ["right_palm","right_ankle"]]
req_body_points[ref_body_dict[ref_datum[0]]]

(744, 138)

In [13]:
#Finding Distance between right palm and ankle
import cv2
img=cv2.imread("Comparision with ground truth\yogframe_dimyog.jpg")
xyp=[]
xyp2=[]
for i in ref_datum:
    p=ref_body_dict[i]
    a=body_points[p][1:]
    b=req_body_points[p]
    xyp.append(a)
    xyp2.append(b)

# for i in range(len(xyp)):
#     print(xyp[i][0],xyp[i+1][0])
#     cv2.line(img,xyp[i][0],xyp[i][1],(0,255,0),2)
cv2.line(img,xyp[0],xyp[1],(0,255,0),2)
#     distance=str(l_dist(xyp[i][0][0],xyp[i][0][1],xyp[i][1][0],xyp[i][1][1]))
distance=str(l_dist(xyp[0][0],xyp[0][1],xyp[1][0],xyp[1][1]))
p=int((xyp[0][0]+xyp[1][0])/2)
q=int((xyp[0][1]+xyp[1][1])/2)
cv2.putText(img,distance,(p,q),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2,cv2.LINE_AA)


cv2.line(img,xyp2[0],xyp2[1],(0,255,0),2)
#     distance=str(l_dist(xyp[i][0][0],xyp[i][0][1],xyp[i][1][0],xyp[i][1][1]))
distance=str(l_dist(xyp2[0][0],xyp2[0][1],xyp2[1][0],xyp2[1][1]))
p=int((xyp2[0][0]+xyp2[1][0])/2)
q=int((xyp2[0][1]+xyp2[1][1])/2)
cv2.putText(img,distance,(p,q),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2,cv2.LINE_AA)
# cv2.imshow('Ima',img)
# # # # cv2.imwrite("Comparision with ground truth\yogframe_dimyog_ptsdist.jpg",img)
# cv2.waitKey()
# cv2.destroyAllWindows()

array([[[119, 119, 119],
        [119, 119, 119],
        [119, 119, 119],
        ...,
        [125, 124, 126],
        [125, 124, 126],
        [125, 124, 126]],

       [[119, 119, 119],
        [119, 119, 119],
        [119, 119, 119],
        ...,
        [125, 124, 126],
        [125, 124, 126],
        [125, 124, 126]],

       [[119, 119, 119],
        [119, 119, 119],
        [119, 119, 119],
        ...,
        [125, 124, 126],
        [125, 124, 126],
        [125, 124, 126]],

       ...,

       [[128, 123, 124],
        [128, 123, 124],
        [128, 123, 124],
        ...,
        [126, 121, 123],
        [126, 121, 123],
        [126, 121, 123]],

       [[128, 123, 124],
        [128, 123, 124],
        [128, 123, 124],
        ...,
        [126, 121, 122],
        [126, 121, 123],
        [125, 120, 122]],

       [[128, 123, 124],
        [128, 123, 124],
        [128, 123, 124],
        ...,
        [126, 121, 122],
        [125, 120, 122],
        [125, 120, 122]]

In [14]:
ref_datum=[i for i in ref_body_dict.keys() if i in ["right_ankle","left_ankle"]]
req_body_points[ref_body_dict[ref_datum[0]]]

(797, 645)

In [15]:
#Finding Distance between right and left ankle
xyp=[]
xyp2=[]
for i in ref_datum:
    p=ref_body_dict[i]
    a=body_points[p][1:]
    b=req_body_points[p]
    xyp.append(a)
    xyp2.append(b)

# for i in range(len(xyp)):
#     print(xyp[i][0],xyp[i+1][0])
#     cv2.line(img,xyp[i][0],xyp[i][1],(0,255,0),2)
cv2.line(img,xyp[0],xyp[1],(0,255,0),2)
#     distance=str(l_dist(xyp[i][0][0],xyp[i][0][1],xyp[i][1][0],xyp[i][1][1]))
distance=str(l_dist(xyp[0][0],xyp[0][1],xyp[1][0],xyp[1][1]))
p=int((xyp[0][0]+xyp[1][0])/2)
q=int((xyp[0][1]+xyp[1][1])/2)
cv2.putText(img,distance,(p,q),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2,cv2.LINE_AA)


cv2.line(img,xyp2[0],xyp2[1],(0,255,0),2)
#     distance=str(l_dist(xyp[i][0][0],xyp[i][0][1],xyp[i][1][0],xyp[i][1][1]))
distance=str(l_dist(xyp2[0][0],xyp2[0][1],xyp2[1][0],xyp2[1][1]))
p=int((xyp2[0][0]+xyp2[1][0])/2)
q=int((xyp2[0][1]+xyp2[1][1])/2)
cv2.putText(img,distance,(p,q),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2,cv2.LINE_AA)

str1="legs_dist & Rleg_palm_dstratio Dim=0.76"
str2="legs_dist & Rleg_palm_dstratio Inf=0.62"
cv2.putText(img,str1,(15,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1,cv2.LINE_AA)
cv2.putText(img,str2,(15,70),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1,cv2.LINE_AA)
# cv2.imwrite("Comparision with ground truth\yogframe_dimyog_refptsdist_ratio.jpg",img)
# cv2.imshow('Ima',img)
# cv2.waitKey()
# cv2.destroyAllWindows()

array([[[119, 119, 119],
        [119, 119, 119],
        [119, 119, 119],
        ...,
        [125, 124, 126],
        [125, 124, 126],
        [125, 124, 126]],

       [[119, 119, 119],
        [119, 119, 119],
        [119, 119, 119],
        ...,
        [125, 124, 126],
        [125, 124, 126],
        [125, 124, 126]],

       [[119, 119, 119],
        [119, 119, 119],
        [119, 119, 119],
        ...,
        [125, 124, 126],
        [125, 124, 126],
        [125, 124, 126]],

       ...,

       [[128, 123, 124],
        [128, 123, 124],
        [128, 123, 124],
        ...,
        [126, 121, 123],
        [126, 121, 123],
        [126, 121, 123]],

       [[128, 123, 124],
        [128, 123, 124],
        [128, 123, 124],
        ...,
        [126, 121, 122],
        [126, 121, 123],
        [125, 120, 122]],

       [[128, 123, 124],
        [128, 123, 124],
        [128, 123, 124],
        ...,
        [126, 121, 122],
        [125, 120, 122],
        [125, 120, 122]]

In [17]:
#Angle between points
import math
def gradient(pt1,pt2):
    return (pt2[1]-pt1[1])/(pt2[0]-pt1[0])

def getangle(pointslist):
    pt1,pt2,pt3=pointslist[-3:]
    m1=gradient(pt1,pt2)
    m2=gradient(pt1,pt3)
    angR = math.atan((m2-m1)/(1+(m2*m1)))
    angD = round(math.degrees(angR))
    return angD

In [18]:
ref_datum=[i for i in ref_body_dict.keys() if i in ['right_shoulder','right_hip','right_palm']]
req_body_points[ref_body_dict[ref_datum[0]]]

(748, 328)

In [19]:
right_shoulder=req_body_points[ref_body_dict[ref_datum[0]]]
right_hip=req_body_points[ref_body_dict[ref_datum[1]]]
right_palm=req_body_points[ref_body_dict[ref_datum[2]]]
ang_points=[right_shoulder,right_hip,right_palm]

In [20]:
ang_points=[right_shoulder,right_hip,right_palm]
angD=getangle(ang_points)
print(angD)


68


In [21]:
import cv2
img=cv2.imread("Comparision with ground truth\yogframe_dimyog.jpg")
cv2.putText(img,str(angD)+" deg",right_shoulder,cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1,cv2.LINE_AA)
cv2.imshow('img',img)

cv2.waitKey()
cv2.destroyAllWindows()

In [22]:
ref_datum=[i for i in ref_body_dict.keys() if i in ['right_shoulder','right_hip','right_palm']]
body_points[ref_body_dict[ref_datum[0]]][1:]

(792, 353)

In [23]:
right_shoulder=body_points[ref_body_dict[ref_datum[0]]][1:]
right_hip=body_points[ref_body_dict[ref_datum[1]]][1:]
right_palm=body_points[ref_body_dict[ref_datum[2]]][1:]
ang_points=[right_shoulder,right_hip,right_palm]

ang_points=[right_shoulder,right_hip,right_palm]
angD=getangle(ang_points)
# print(angD)

cv2.putText(img,str(angD)+" deg",right_shoulder,cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),1,cv2.LINE_AA)
cv2.imshow('img',img)
cv2.imwrite("Comparision with ground truth\yogframe_dimyog_angles.jpg",img)
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
import cv2
img1=cv2.imread("images\dim_triangles-cropped.png")
img2=cv2.imread("images\yogframe_cropped.jpg")
img1.shape,img2.shape

((480, 396, 3), (618, 478, 3))

In [7]:
import cv2
img1=cv2.imread("images\dim_triangles-cropped.png")
img2=cv2.imread("images\yogframe_cropped.jpg")
img1.shape,img2.shape
img1=cv2.resize(img1,(int(618/480)*396,618))
print(img1.shape,img2.shape)
# cv2.imwrite("images/dim_pose_sameht.jpg",img1)
cv2.imshow('img',img1)
cv2.waitKey()
cv2.destroyAllWindows()


(618, 396, 3) (618, 478, 3)


In [9]:
import cv2
img=cv2.imread('images\dim_image_org.png')
cropped_image = img[65:545,50:435+80]
cv2.imshow('img',cropped_image)
cv2.imshow('org',img)
# cv2.imwrite('Images_output\dimimg_crop.jpg',cropped_image)
cv2.waitKey()
cv2.destroyAllWindows()

In [10]:
import cv2
img1=cv2.imread('Images_output\dimimg_crop.jpg')
img2=cv2.imread('images\yogframe_cropped.jpg')
# cv2.imshow('dim',img1)
# cv2.imshow('inf',img2)

# cv2.imwrite('Images_output\dimimg_crop.jpg',cropped_image)
# cv2.waitKey()
# cv2.destroyAllWindows()
img1.shape,img2.shape,img1.shape[1::-1]

((480, 419, 3), (618, 478, 3), (419, 480))

In [28]:
import cv2
import numpy as np

img1=cv2.imread('Images_output\dimimg_crop.jpg')
img2=cv2.imread('images\yogframe_cropped.jpg')

# img2 = img2[0:480,0:425]
img2= cv2.resize(img2,img1.shape[1::-1])

added_image = cv2.addWeighted(img1,0.7,img2,0.3,0)


cv2.imshow('Blended Image',added_image)
# cv2.imwrite("Trial&error for correct overlay\sample_dim_dimresized2.jpg",added_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
import cv2
img2=cv2.imread('images\yogframe_cropped.jpg')
cropped_img2 = img2[0:480,0:425]
print(cropped_img2.shape)
cv2.imshow('req frame Image',cropped_img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

(480, 425, 3)


0. Assumptions:
dimensions image W 2x user image W;
H is the same for both;
user image has fixed W and H

Get the MRP from the user image - left foot in this case
On the user image, D will be MRP.W;
On the dimensions image, crop LEFT SIDE by D from the MRP of the dim image
dim.MRP.W = 300 _____ (300-d:user.W-300)
On dimensions image crop RightSide by user.W

In [ ]:
# Get the MRP from the user image - left foot in this case